In [ ]:
# ===========================================
# 📦 Install dependencies(only need to run once)
# ===========================================
!pip install -q openai networkx matplotlib jieba rich
# ===========================================
# 📦 Install dependencies ()
# ===========================================
!pip install -q --upgrade openai chardet networkx matplotlib jieba rich

In [ ]:
# # ============================================================
# # 📚 Batch correct garbled novel text → UTF-8 (Colab )
# # ============================================================

# # ① —— path ()
# INPUT_DIR = "/content/garbled_novels" # Novel
# OUTPUT_DIR = "/content/novels" # output
# OVERWRITE = False # True = INPUT_DIR

# # ② —— Install dependencies
# !pip -q install chardet

# # ③ ——
# import os, re, logging, unicodedata, shutil
# from pathlib import Path
# import chardet

# logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

# # ---------- ----------
# COMMON_ENCODINGS = [
# "utf-8",
# "gb18030", "gbk",
# "big5",
# "utf-16le", "utf-16be",
# ]

# def detect_encoding(raw: bytes) -> str | None:
# """ chardet，"""
# info = chardet.detect(raw)
# enc, conf = (info["encoding"] or "").lower(), info["confidence"] or 0
# if enc and conf > 0.8:
# return enc
# for e in COMMON_ENCODINGS:
# try:
# raw.decode(e)
# return e
# except UnicodeDecodeError:
# continue
# return None

# def safe_title(name: str) -> str:
# """//，"""
# name = unicodedata.normalize("NFKC", name)
# name = re.sub(r"[\r\n\t]+", " ", name)
# name = re.sub(r'[\\/:*?"<>|]', "_", name)
# name = re.sub(r"\s+", "", name)
# return name[:80] or ""

# def convert_one(src: Path, dst_root: Path, *, overwrite: bool = False) -> bool:
# raw = src.read_bytes()
# enc = detect_encoding(raw)
# if not enc:
# logging.warning(f"[] → {src.as_posix()}")
# return False
# try:
# text = raw.decode(enc, errors="ignore")
# except UnicodeDecodeError as e:
# logging.warning(f"[] ({enc}: {e}) → {src.as_posix()}")
# return False

# rel = src.relative_to(INPUT_DIR)
# #
# clean_parts = [safe_title(p.stem) + p.suffix if p.is_file()
# else safe_title(p.name)
# for p in rel.parents[::-1]] # include intermediate dirs
# # path
# dst_path = dst_root.joinpath(*clean_parts[::-1])
# dst_path.parent.mkdir(parents=True, exist_ok=True)

# if dst_path.exists() and not overwrite:
# logging.info(f"[] → {dst_path.as_posix()}")
# return False

# dst_path.write_text(text, encoding="utf-8")
# logging.info(f"[OK] {src.name} ({enc}) → {dst_path.as_posix()}")
# return True

# # ---------- ----------
# src_dir = Path(INPUT_DIR).expanduser().resolve()
# dst_dir = src_dir if OVERWRITE else Path(OUTPUT_DIR).expanduser().resolve()
# if not src_dir.is_dir():
# raise ValueError(f"{src_dir} ")

# success = 0
# for txt in src_dir.rglob("*.txt"):
# success += convert_one(txt, dst_dir, overwrite=OVERWRITE)

# print(f"\n✅ {success} → {dst_dir}")

In [ ]:
# ============================================================
# 📚 Full pipeline modules(DeepSeek + OpenRouter/Claude 3.7 dual backends)
# 1. Split chapters split_novels()
# 2. LLM process() → *_processed.txt
# 3. Extension：provider "deepseek" / "openrouter"
# - response_format={"type":"json_object"}
# ============================================================

# ── ────────────────────────────────────────────────
!pip -q install --upgrade openai tqdm chardet pandas matplotlib jieba networkx requests

# ── ───────────────────────────────────────────────────
import os, re, json, logging, unicodedata, chardet, requests
from pathlib import Path
from typing import List, Dict, Tuple
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

# ── DeepSeek / OpenRouter ───────────────────────────────
class Provider(str):
    DEEPSEEK   = "deepseek"
    OPENROUTER = "openrouter"

DEEPSEEK_API_KEY  = os.getenv("DEEPSEEK_API_KEY")   or "<YOUR_API_KEY>"
DEEPSEEK_URL      = "https://api.deepseek.com"
DEEPSEEK_MODEL    = "deepseek-chat"

OPENROUTER_API_KEY= os.getenv("OPENROUTER_API_KEY") or "<YOUR_API_KEY>"
OPENROUTER_URL    = "https://openrouter.ai/api/v1/chat/completions"
OPENROUTER_MODEL  = "anthropic/claude-3.7-sonnet"

import openai                                # DeepSeek
deep_client = openai.OpenAI(api_key=DEEPSEEK_API_KEY, base_url=DEEPSEEK_URL)

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

# ============================================================
# 1️⃣ Split chapters split_novels()
# ============================================================
_CHAPTER_PAT = re.compile(
    r"""
    ^\s*(
        第[\d零一二三四五六七八九十百千万]+\s*[章节卷回]\s* |
        [零一二三四五六七八九十百千万]{1,4}[\.．、\s]+ |
        (?:Chapter|CHAPTER)\s+\d+               |
        \d{1,3}[\.．、]\s*                      |
        \d{1,3}\s+
    )\s*(.*?)$
    """, re.MULTILINE | re.IGNORECASE | re.VERBOSE
)

def _safe_name(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = re.sub(r"[\r\n\t]+", " ", s)
    s = re.sub(r'[\\/:*?"<>|]', "_", s)
    s = re.sub(r"\s+", "", s)
    return s[:80] or "未知"

def _auto_decode(path: Path) -> str:
    raw = path.read_bytes()
    enc = chardet.detect(raw)["encoding"] or "utf-8"
    return raw.decode(enc, errors="ignore")

def split_novels(input_dir: str, output_base: str | None = None) -> Dict[str, List[Path]]:
    in_p, out_p = Path(input_dir), Path(output_base or f"{input_dir.rstrip('/')}_chapters")
    out_p.mkdir(parents=True, exist_ok=True)

    novels: Dict[str, List[Path]] = {}
    for txt in in_p.glob("*.txt"):
        book_dir = out_p / _safe_name(txt.stem); book_dir.mkdir(exist_ok=True)
        data = _auto_decode(txt)
        ms = list(_CHAPTER_PAT.finditer(data))
        blocks = [(ms[i].group().strip(),
                   data[ms[i].end():ms[i+1].start()] if i+1<len(ms) else data[ms[i].end():])
                  for i in range(len(ms))] if ms else \
                 [(f"未知章节{i+1}", data[s:s+5000]) for i,s in enumerate(range(0,len(data),5000))]
        paths=[]
        for i,(title,body) in enumerate(blocks,1):
            p=book_dir/f"{i:03d}_{_safe_name(title)}.txt"
            p.write_text(body.strip(),encoding="utf-8"); paths.append(p)
        novels[book_dir.name]=paths
        logging.info(f"《{txt.stem}》→ {len(paths)} 章")
    return novels
# ============================================================
# 2️⃣ LLM _call_llm()
# ============================================================
def _call_llm(prompt:str, content:str,
              *, provider:Provider=Provider.DEEPSEEK,
              json_mode:bool=False) -> str:
    if provider==Provider.DEEPSEEK:
        rsp = deep_client.chat.completions.create(
            model=DEEPSEEK_MODEL,
            messages=[{"role":"system","content":prompt},
                      {"role":"user","content":content}],
            temperature=0.3,
            max_tokens=4096,
            response_format={"type":"json_object"} if json_mode else None
        )
        return rsp.choices[0].message.content.strip()

    if provider==Provider.OPENROUTER:
        body = {
            "model": OPENROUTER_MODEL,
            "messages":[
                {"role":"system","content":prompt},
                {"role":"user","content":content}
            ]
        }
        if json_mode:
            body["response_format"]={"type":"json_object"}
        r = requests.post(OPENROUTER_URL,
                          headers={
                              "Authorization":f"Bearer {OPENROUTER_API_KEY}",
                              "Content-Type":"application/json"
                          },
                          data=json.dumps(body))
        if r.status_code!=200:
            raise RuntimeError(f"OpenRouter {r.status_code}: {r.text[:200]}")
        return r.json()["choices"][0]["message"]["content"].strip()

    raise ValueError("Unknown provider")

# ============================================================
# 3️⃣ process_chapters() / process()
# ============================================================
def process_chapters(chapter_files: List[Path],
                     prompt: str,
                     *,
                     provider:Provider = Provider.DEEPSEEK,
                     json_mode: bool = True,
                     workers:int = 4,
                     suffix="_processed.txt"):
    def _run(fp:Path):
        res=_call_llm(prompt,fp.read_text(encoding="utf-8"),
                      provider=provider,json_mode=json_mode)
        out=fp.with_name(fp.stem+suffix); out.write_text(res,encoding="utf-8")
        return out
    with ThreadPoolExecutor(max_workers=workers) as ex:
        list(ex.map(_run, chapter_files))

def _gather(root:Path)->List[Path]:
    return sorted(root.rglob("*.txt"))

def process(chapter_root:str,*,
            prompt:str,
            provider:Provider=Provider.DEEPSEEK,
            out_dir:str="/content/json_results",
            json_mode:bool=True,
            chapters:Tuple[int,int]|None=None):
    s,e = chapters or (1,float("inf"))
    root, out_base = Path(chapter_root), Path(out_dir); out_base.mkdir(parents=True,exist_ok=True)
    books = [d for d in root.iterdir() if d.is_dir()] or [root]

    for book in tqdm(books, desc="📚 书库"):
        files=[p for p in _gather(book) if s<=int(p.stem.split("_")[0])<=e]
        for group_idx in range(0,len(files),10):
            grp=files[group_idx:group_idx+10]
            if not grp: continue
            start=int(grp[0].stem[:3]); end=start+len(grp)-1
            sub=out_base/book.name/f"{start:03d}-{end:03d}"; sub.mkdir(parents=True,exist_ok=True)
            todo=[p for p in grp if not (sub/f"{p.stem}_processed.txt").exists()]
            if todo:
                process_chapters(todo,prompt,
                                 provider=provider,json_mode=json_mode)
                for tp in todo:
                    (sub/f"{tp.stem}_processed.txt").write_text(
                        (tp.parent/f"{tp.stem}_processed.txt").read_text(encoding="utf-8"),
                        encoding="utf-8")
                    (tp.parent/f"{tp.stem}_processed.txt").unlink()
    print(f"✅ 输出目录：{out_base}")


def run_analysis(
    chapter_root: str,
    *,
    prompt: str,
    provider: Provider = Provider.DEEPSEEK,
    out_dir: str = "/content/json_results",
    mode: str | tuple[int, int] = "full",   # ←
    json_mode: bool = True,
):
    """
    mode 用法
    --------
    • "full"          → 分析整本
    • 100             → 只分析 1~100 章
    • (51, 150)       → 分析 51~150 章
    """
    if mode == "full":
        chapters = None
    elif isinstance(mode, int):
        chapters = (1, mode)
    elif isinstance(mode, tuple) and len(mode) == 2:
        chapters = mode
    else:
        raise ValueError("mode 必须是 'full'、整数 N，或 (start, end) 元组")

    process(
        chapter_root=chapter_root,
        prompt=prompt,
        provider=provider,
        out_dir=out_dir,
        json_mode=json_mode,
        chapters=chapters
    )

In [ ]:
# # 1. Split chapters
# split("/content/novels", "/content/novels_chapters")

# # 2. Chapter Prompt(， prompt_chapter_json)
# PROMPT_JSON = r"""
# 。【Chapter】， **1000-2000 ** Chapter， ** JSON ** (output Markdown)。

# JSON ：
# {
# "Chapter": "<100-200 ，Chapter>",
# "": "<200-300 ，Chapter>",
# "": "<200-300 ，Chapter>",
# "": "<200-400 ，Chapter>",
# "": "<200-300 ，Chapter>",
# "": "<100-200 ，Chapter>",
# "": "<100-200 ，Chapter>",
# "": ["A", "B", ...],
# "": ["1", "2", ...],
# "": ["1", "2"],
# "_": ["1 ", "2 "],
# "_": ["A ", ...]
# }

# ：
# 1. ；。
# 2. 12 ，。
# 3. “ / / / ” JSON 。
# 4. ≤10%，。
# 【Chapter】：
# """
# # 3. ( 1–100 )
# # 1–100
# process(
# chapter_root="/content/novels_chapters",
# prompt=prompt_chapter_json,
# out_dir="/content/json_results",
# json_mode=True,
# chapters=(1, 100)
# )

# # ()
# # process(
# # chapter_root="/content/novels_chapters",
# # prompt=prompt_chapter_json,
# # out_dir="/content/json_results",
# # json_mode=True
# # )

# # 4.
# # stat_library("/content/json_results", out_root="/content/novel_stats")

In [ ]:
# ============================================================
# 📊 NovelChapter JSON statistics(，interval compression + save)
# ============================================================
import json, re
from pathlib import Path
from collections import defaultdict
from typing import Dict, List, Tuple

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# ---------- ----------
_DIGIT_RE = re.compile(r"(\d{3})")
def _extract_no(p: Path) -> int:
    m = _DIGIT_RE.search(p.stem) or re.match(r"(\d{3})-", p.parent.name)
    return int(m.group(1)) if m else -1

def _compress(nums: List[int]) -> List[str]:
    if not nums: return []
    nums = sorted(nums)
    res, s, prev = [], nums[0], nums[0]
    for n in nums[1:]:
        if n == prev + 1: prev = n; continue
        res.append(f"{s}-{prev}" if s != prev else f"{s}")
        s = prev = n
    res.append(f"{s}-{prev}" if s != prev else f"{s}")
    return res

# ---------- ----------
# ---------- stat_book ----------
# ---------- ： ----------
def _map_occ(df: pd.DataFrame, col: str) -> Dict[str, List[int]]:
    mp = defaultdict(list)
    for _, row in df.iterrows():
        for item in row[col]:
            mp[item].append(row["chapter"])
    return mp

def _build(mp: Dict[str,List[int]], *, min_chaps=1, top_n=20):
    data = [
        {"name": k, "count": len(v), "chapters": _compress(v)}
        for k, v in mp.items() if len(v) >= min_chaps
    ]
    return sorted(data, key=lambda x: x["count"], reverse=True)[:top_n]

# ---------- ----------
def stat_book(book_dir: str | Path,
              *,
              min_chapters: int = 3,
              top_n: int = 30,
              show_plot: bool = False) -> dict:
    """对单本小说目录生成统计 dict"""
    book_dir = Path(book_dir)
    rows, auto_idx = [], 1

    for p in sorted(book_dir.rglob("*_processed.txt")):
        chap_no = _extract_no(p)
        if chap_no == -1:
            chap_no, auto_idx = auto_idx, auto_idx + 1
        try:
            data = json.loads(p.read_text(encoding="utf-8"))
        except json.JSONDecodeError:
            print(f"⚠️ 跳过无效 JSON: {p}"); continue

        rows.append({
            "chapter":   chap_no,
            "characters": data.get("出现人物", []),
            "scenes":     data.get("出现场景", []),
            "props":      data.get("出现道具", []),
            "setup":      data.get("伏笔_设下", []),
            "recycle":    data.get("伏笔_回收", []),
        })

    if not rows:
        raise ValueError(f"{book_dir} 无有效 *_processed.txt")

    df = pd.DataFrame(rows).sort_values("chapter").reset_index(drop=True)  # ←

    char_map = _map_occ(df, "characters")
    major    = {k: v for k, v in char_map.items() if len(v) >= min_chapters}

    summary = {
        "主要角色": _build(major, min_chaps=min_chapters, top_n=top_n),
        "场景":     _build(_map_occ(df, "scenes"),  top_n=top_n),
        "道具":     _build(_map_occ(df, "props"),   top_n=top_n),
        "伏笔_设下": _build(_map_occ(df, "setup"),  top_n=top_n),
        "伏笔_回收": _build(_map_occ(df, "recycle"),top_n=top_n),
    }

    if show_plot and major:
        plt.figure(figsize=(10, 4))
        for c, chs in major.items():
            plt.plot(df["chapter"], [1 if n in chs else 0 for n in df["chapter"]], label=c)
        plt.xlabel("章节号"); plt.ylabel("出现(1)"); plt.title(book_dir.name); plt.legend()
        plt.show()

    return summary

# ---------- ----------
def stat_library(root_dir: str | Path,
                 out_root: str | Path = "/content/novel_stats",
                 *, min_chapters: int = 3,
                 top_n: int = 10,
                 show_plot: bool = False):
    root_dir = Path(root_dir); out_root = Path(out_root)
    out_root.mkdir(parents=True, exist_ok=True)

    for book in root_dir.iterdir():
        if not book.is_dir(): continue
        try:
            res = stat_book(book,
                            min_chapters=min_chapters,
                            top_n=top_n,
                            show_plot=show_plot)
        except ValueError as e:
            print(e); continue

        out_file = out_root / book.name / f"{book.name}_stats.json"
        out_file.parent.mkdir(parents=True, exist_ok=True)
        out_file.write_text(json.dumps(res, ensure_ascii=False, indent=2), encoding="utf-8")
        print(f"✅ 写入 {out_file.relative_to(out_root)}")

In [ ]:
# ============================================================
# 📦 MODULE CELL — merge_utils.py (module，)
# ============================================================
!pip -q install --upgrade openai tqdm

import os, re, json
from pathlib import Path
from collections import defaultdict
from typing import List, Dict
from tqdm.auto import tqdm
from openai import OpenAI

# ---------- ----------
def is_chunk_dir(name: str) -> bool:
    """001-010 形式"""
    return bool(re.fullmatch(r"\d{3}-\d{3}", name))

def call_deepseek(text: str, *, api_key: str, base_url: str, model: str) -> str:
    client = OpenAI(api_key=api_key, base_url=base_url)
    prompt = (
        "你是一位资深小说分析师。请根据接下来提供的十章合并文本，"
        "输出每位角色在这十章内的【成长轨迹】【等级/实力变化】【关键事件】。"
        "返回 JSON，键包括：characters(数组，每项含 name, growth, level_change, key_events)。"
    )
    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": prompt},
                  {"role": "user", "content": text}],
        temperature=0.3,
        max_tokens=4096,
        response_format={"type": "json_object"}
    )
    return resp.choices[0].message.content.strip()

def auto_group(files: List[Path]) -> Dict[str,List[Path]]:
    """平铺章节 -> {'001-010':[files], ...}"""
    groups = defaultdict(list)
    for f in files:
        m = re.match(r"(\d{3})_", f.stem)
        idx = int(m.group(1)) if m else 0
        st = ((idx - 1)//10)*10 + 1
        ed = st + 9
        groups[f"{st:03d}-{ed:03d}"].append(f)
    return groups

# ---------- ----------
def run_pipeline(root_dir: str | Path,
                 merge_dir: str | Path,
                 api_key: str,
                 *,
                 base_url: str = "https://api.deepseek.com",
                 model: str = "deepseek-chat"):
    """
    1. 合并每 10 章 *_processed.txt → merge_dir/书名/区间.txt
    2. DeepSeek 分析 → 同目录 区间_analysis.json
    3. 已存在分析文件自动跳过
    """
    root   = Path(root_dir).expanduser()
    merge  = Path(merge_dir).expanduser(); merge.mkdir(exist_ok=True)
    for book in tqdm([d for d in root.iterdir() if d.is_dir()], desc="📚 书库"):
        # a)
        chunks = [c for c in book.iterdir() if c.is_dir() and is_chunk_dir(c.name)]
        # b) ->
        if not chunks:
            files = sorted(book.glob("*_processed.txt"))
            for name, lst in auto_group(files).items():
                tmp = book / name; tmp.mkdir(exist_ok=True)
                for f in lst:
                    link = tmp / f.name
                    if not link.exists():
                        os.symlink(f, link)
                chunks.append(tmp)

        # -- 10 --
        for ck in tqdm(sorted(chunks), desc=f"  📂 {book.name}", leave=False):
            mtxt = merge / book.name / f"{ck.name}.txt"
            mjs  = merge / book.name / f"{ck.name}_analysis.json"
            mtxt.parent.mkdir(parents=True, exist_ok=True)
            if mjs.exists():
                continue
            parts = sorted(ck.glob("*_processed.txt"))
            if not parts:
                continue
            mtxt.write_text("\n\n".join(p.read_text(encoding='utf-8') for p in parts), encoding='utf-8')
            try:
                res = call_deepseek(mtxt.read_text(encoding='utf-8'),
                                    api_key=api_key, base_url=base_url, model=model)
                json.loads(res); mjs.write_text(res, encoding='utf-8')
                tqdm.write(f"✅ {mjs.relative_to(merge)}")
            except Exception as e:
                tqdm.write(f"❌ DeepSeek 失败 {ck.name}: {e}")

    print(f"🎉 处理完成: 合并 & 分析文件 → {merge_dir}")

# ---------- Cell ----------
__all__ = ["run_pipeline"]

In [ ]:
# ============================================================
# 📦 MODULE CELL — outline_batch.py (10 + 100 )
# ============================================================
from pathlib import Path
from collections import defaultdict
from typing   import List, Dict
import os, re, json, sys, types
from tqdm.auto import tqdm

__all__ = ["export_outlines"]

# ---------- ----------
_DIGIT_RE  = re.compile(r"(\d{3})_")      # 000
_CHUNK_RE  = re.compile(r"\d{3}-\d{3}")   # 001-010
_FIELDS    = ("情节摘要导语", "章节定位导语")

# ---------- ----------
def _is_chunk_dir(name:str)->bool: return bool(_CHUNK_RE.fullmatch(name))

def _auto_group10(files)->Dict[str,List[Path]]:
    """平铺章节 → {'001-010':[...], ...}"""
    mp=defaultdict(list)
    for f in files:
        m=_DIGIT_RE.match(f.stem); idx=int(m.group(1)) if m else 0
        st=((idx-1)//10)*10+1; ed=st+9
        mp[f"{st:03d}-{ed:03d}"].append(f)
    return {k:sorted(v) for k,v in mp.items()}

def _take_outline(d:dict, fields)->str:
    for f in fields:
        if f in d and d[f]: return str(d[f]).strip()
    return "（未找到细纲字段）"

# ---------- 10 → ----------
def _collect_chunks(book:Path, fields)->Dict[str,str]:
    """返回 {10区间名: outline_text}"""
    # 10 ：
    chunks=[c for c in book.iterdir() if c.is_dir() and _is_chunk_dir(c.name)]
    if not chunks:
        for name,lst in _auto_group10(sorted(book.glob("*_processed.txt"))).items():
            tmp=book/name; tmp.mkdir(exist_ok=True)
            for f in lst: (tmp/f.name).symlink_to(f)
            chunks.append(tmp)

    outlines={}
    for ck in sorted(chunks):
        lines=[f"\n########## {ck.name} ##########\n"]
        for fp in sorted(ck.glob("*_processed.txt")):
            try: data=json.loads(fp.read_text(encoding="utf-8"))
            except json.JSONDecodeError: continue
            no=_DIGIT_RE.match(fp.stem); chap=no.group(1) if no else "???"
            title=fp.stem[len(chap)+1:]
            lines.append(f"《{book.name}》 第{chap}章  {title}")
            lines.append(_take_outline(data, fields)); lines.append("")
        outlines[ck.name]="\n".join(lines)
    return outlines

# ---------- ----------
def export_outlines(root_dir:str|Path="/content/json_results",
                    out_root:str|Path="/content/outline_summaries",
                    *, fields=_FIELDS):
    root, out_root = Path(root_dir), Path(out_root)
    if not root.exists(): raise FileNotFoundError(root)

    for book in tqdm([d for d in root.iterdir() if d.is_dir()], desc="📚 书库"):
        chunk_texts=_collect_chunks(book, fields)          # 10
        # --- 10 ---
        out10 = out_root/book.name/f"{book.name}_outline.txt"
        out10.parent.mkdir(parents=True, exist_ok=True)
        out10.write_text("\n".join(chunk_texts.values()), encoding="utf-8")
        tqdm.write(f"✅ 10章汇总 → {out10.relative_to(out_root)}")

        # --- 100 ---
        big_lines=[]; sorted_keys=sorted(chunk_texts)
        for i in range(0,len(sorted_keys),10):
            group=sorted_keys[i:i+10]
            if not group: continue
            start,end=group[0].split("-")[0], group[-1].split("-")[1]
            big_lines.append(f"\n============ {start}-{end} ============\n")
            for k in group: big_lines.append(chunk_texts[k])
        out100 = out_root/book.name/f"{book.name}_outline_100.txt"
        out100.write_text("\n".join(big_lines), encoding="utf-8")
        tqdm.write(f"✅ 100章汇总 → {out100.relative_to(out_root)}")

# ---------- ----------
mod=types.ModuleType("outline_batch"); mod.export_outlines=export_outlines
sys.modules["outline_batch"]=mod

In [ ]:
# from pathlib import Path
# import json, textwrap
# from tqdm.auto import tqdm

# def batch_validate(root_dir: str | Path,
# pattern: str = "*_processed.txt",
# verbose: bool = True) -> list[str]:
# """
# Validate root_dir pattern JSON
# error_file ； verbose=True
# """
# root = Path(root_dir)
# files = sorted(root.rglob(pattern))
# bad = []

# for fp in tqdm(files, desc="🔍 JSON Validate"):
# txt = fp.read_text(encoding="utf-8", errors="ignore")
# try:
# json.loads(txt)
# except json.JSONDecodeError as e:
# bad.append(fp.as_posix())
# if verbose:
# pos = e.pos
# snippet = textwrap.shorten(txt[max(0,pos-40):pos+40], width=120, placeholder="…")
# print(f"\n✗ {fp}\n {e}\n …{snippet}…\n")
# if not bad:
# print("✅ JSON")
# else:
# print(f"⚠️ {len(bad)} JSON invalid")
# return bad

# # ---------- ----------
# error_files = batch_validate("/content/json_results")

In [ ]:
# # ============================================================
# # 🤖 DeepSeek invalid JSON
# # ============================================================
# !pip -q install --upgrade openai tqdm

# import json, textwrap, os
# from pathlib import Path
# from tqdm.auto import tqdm
# import openai

# # ---------- ----------
# TARGET_DIR = "/content/json_results" # ←
# DEEPSEEK_API_KEY = "<YOUR_API_KEY>" # ← DeepSeek Key
# DEEPSEEK_MODEL = "deepseek-chat"

# client = openai.OpenAI(api_key=DEEPSEEK_API_KEY,
# base_url="https://api.deepseek.com")

# PROMPT_FIX = """
# Chapter JSON，：
# 1. (、、)，
# 2. JSON，
# 3. output
# """

# # ---------- Step 1: & invalid JSON ----------
# bad_files = []
# for fp in tqdm(Path(TARGET_DIR).rglob("*_processed.txt"),
# desc="📄 "):
# try:
# json.loads(fp.read_text(encoding="utf-8"))
# except json.JSONDecodeError:
# bad_files.append(fp)

# print(f"🔍 {len(bad_files)} invalid JSON\n")

# # ---------- Step 2: DeepSeek ----------
# def deepseek_fix(broken_text: str) -> str:
# resp = client.chat.completions.create(
# model=DEEPSEEK_MODEL,
# messages=[
# {"role": "system", "content": PROMPT_FIX},
# {"role": "user", "content": broken_text}
# ],
# temperature=0,
# max_tokens=4096,
# response_format={"type": "json_object"}
# )
# return resp.choices[0].message.content.strip()

# fixed, failed = [], []
# for fp in tqdm(bad_files, desc="🔧 "):
# raw = fp.read_text(encoding="utf-8", errors="ignore")
# try:
# fixed_txt = deepseek_fix(raw)
# json.loads(fixed_txt) #
# fp.write_text(fixed_txt, encoding="utf-8")
# fixed.append(fp)
# except Exception as e:
# failed.append(fp)
# snippet = textwrap.shorten(str(e), width=100, placeholder="…")
# tqdm.write(f"✗ {fp.name} : {snippet}")

# # ---------- ----------
# print(f"\n✅ {len(fixed)} ")
# if failed:
# print(f"⚠️ {len(failed)} ，：")
# for f in failed:
# print(" -", f)

In [94]:
!rm -rf /content/novels_chapters
!rm -rf /content/json_results
!rm -rf /content/novel_stats
!rm -rf /content/merged
!rm -rf /content/garbled_novels
!rm -rf /content/outline_summaries

In [93]:

!rm -rf /content/novel_stats
!rm -rf /content/merged
!rm -rf /content/garbled_novels
!rm -rf /content/outline_summaries

In [ ]:
# 1. Split chapters
split_novels("/content/novels", "/content/novels_chapters")

# 2. Chapter Prompt(， prompt_chapter_json)
prompt_chapter_json = r"""
你是一位专业文学编辑。请阅读我接下来提供的【章节全文】，按照下列要求生成 **1000-2000 字** 的详细章节分析，并 **仅以 JSON 格式** 返回结果（严禁输出任何解释或 Markdown）。

JSON 顶层字段与要求：
{
  "章节定位导语":         "<100-200 字，说明章节在整书中的位置和作用>",
  "情节摘要导语":         "<200-300 字，概括该章节的主要内容,请尽量把剧情讲述完整>",
  "情感与节奏导语":       "<200-300 字，分析章节的情感和节奏变化>",
  "关键场景分析":         "<200-400 字，深入分析章节中的重要场景>",
  "人物角色变化":         "<200-300 字，分析章节中人物的表现和变化>",
  "情感张力变化":         "<100-200 字，分析章节情感的起伏>",
  "节奏与结构观察":       "<100-200 字，分析章节节奏与整书主题和情节的关联>",
  "出现人物":             ["角色A", "角色B", ...],
  "出现道具":             ["道具1", "道具2", ...],
  "出现场景":             ["场景1", "场景2"],
  "伏笔_设下":            ["伏笔1 描述", "伏笔2 描述"],
  "伏笔_回收":            ["伏笔A 回收方式", ...]
}

严格要求：
1. 文本字段须为完整中文段落；字数必须落在区间内。
2. 上述 12 个键一个都不能少，也不能多。
3. “出现人物 / 道具 / 场景 / 伏笔” 用 JSON 数组。
4. 可引用原文≤10%，需加引号注明段落号。
【章节全文】：
"""
# 3. ( 1–100 )
# 1–100
# process(
# chapter_root="/content/novels_chapters",
# prompt=prompt_chapter_json,
# out_dir="/content/json_results",
# json_mode=True,
# chapters=(1, 10)
# )

# # 1) 100 (DeepSeek)
run_analysis(
    chapter_root="/content/novels_chapters",
    prompt=prompt_chapter_json,
    provider=Provider.DEEPSEEK,
    out_dir="/content/json_results",
    mode=(1, 120)
)

# 2) 51 ~ 150 (Claude-3.7 · OpenRouter)
# run_analysis(
# chapter_root="/content/novels_chapters",
# prompt=prompt_chapter_json,
# provider=Provider.OPENROUTER,
# out_dir="/content/json_results",
# mode=(1, 10) #
# )

# # 3) ( "full")
# run_analysis(
# chapter_root="/content/novels_chapters",
# prompt=PROMPT_JSON,
# provider=Provider.DEEPSEEK,
# out_dir="/content/json_ds_full",
# mode="full"
# )

# ()
# process(
# chapter_root="/content/novels_chapters",
# prompt=prompt_chapter_json,
# out_dir="/content/json_results",
# json_mode=True
# )

# 4.
# stat_library("/content/json_results", out_root="/content/novel_stats")

# validate("/content/json_results")


stat_library(
    root_dir="/content/json_results",
    out_root="/content/novel_stats",   # JSON
    min_chapters=3,
    top_n=50,
    show_plot=False                    # True
)

# ============================================================
# 🚀 Usage example CELL
# ============================================================
import sys, types

if "merge_utils" not in sys.modules:
    merge_utils = types.ModuleType("merge_utils")
    for n, obj in globals().items():
        if n in ["run_pipeline", "is_chunk_dir", "auto_group", "call_deepseek"]:
            setattr(merge_utils, n, obj)
    sys.modules["merge_utils"] = merge_utils

# ， reload
merge_utils.run_pipeline(
    root_dir="/content/json_results",
    merge_dir="/content/merged",
    api_key="<YOUR_API_KEY>"
)

import outline_batch as ob

ob.export_outlines(
    root_dir="/content/json_results",          # JSON
    out_root="/content/outline_summaries",     # output
    fields=("情节摘要导语", "章节定位导语")      # detailed outline
)

In [ ]:
# ============================================================
# 🚀 Usage example CELL
# ============================================================
import sys, types

if "merge_utils" not in sys.modules:
    merge_utils = types.ModuleType("merge_utils")
    for n, obj in globals().items():
        if n in ["run_pipeline", "is_chunk_dir", "auto_group", "call_deepseek"]:
            setattr(merge_utils, n, obj)
    sys.modules["merge_utils"] = merge_utils

# ， reload
merge_utils.run_pipeline(
    root_dir="/content/json_results",
    merge_dir="/content/merged",
    api_key="<YOUR_API_KEY>"
)

import outline_batch as ob

ob.export_outlines(
    root_dir="/content/json_results",          # JSON
    out_root="/content/outline_summaries",     # output
    fields=("情节摘要导语", "章节定位导语")      # detailed outline
)

In [ ]:
# 📚 NovelChaptergenerator - Google Colab
# ：outlinedetailed outline，Chapter， OpenAI API NovelChapter(5000)save

import os
import json
import openai
from glob import glob
from datetime import datetime

# ✅ ()
API_KEY = "<YOUR_API_KEY>"  # 🔑 OpenAI API Key
OUTLINE_DIR = "/content/merged/《大道纪》作者_裴屠狗"  # 📁 outline( analysis.json )
DETAIL_DIR = "/content/json_results/《大道纪》作者_裴屠狗/001-006"    # 📁 detailed outline(detailed outline txt )

# BOOK_TITLE = "《》_ chatgpt-4o-latest 0.8" # 📕 ，output
# gpt4.1，chatgpt-4o-latest 0.3，。

# output chatgpt-4o-latest 0.8 gpt-4.1-2025-04-14 o3-2025-04-16
# BOOK_TITLE = "《》_ chatgpt-4o-latest 08" # 📕 ，output
# BOOK_TITLE = "《》_ chatgpt-4o-latest 09" # 📕 ，output
BOOK_TITLE = "《大道纪》作者_裴屠狗 chatgpt-4o-latest 03"       # 📕 ，output
# BOOK_TITLE = "《》_ gpt-4.1-2025-04-14 03" # 📕 ，output
# BOOK_TITLE = "《》_ gpt-4.1-2025-04-14 08" # 📕 ，output
# BOOK_TITLE = "《》_ gpt-4.1-2025-04-14 05" # 📕 ，output
# BOOK_TITLE = "，outline" # 📕 ，output



CHAPTER_RANGE = list(range(1, 20))  # 📌 Chapter， [1,2,3,5]
PROMPT_TEMPLATE = """
你是一位擅长网络小说创作的作家，请根据以下内容创作第{chapter_num}章内容，丝毫不偏离细纲和大纲的内容。仔细检查，不能有任何突兀的句子。请严格遵循结尾不要有总结的句子，这只是一部百万字网文的一个章节。思考大纲，如何连贯开头和结尾。
要求风格为网文风格，语言要有感染力但不过于浮夸，通过行动而不是旁白语言描写表达主角的心理活动，避开“太小白”或“太装逼”的两极，既有代入感也要有些内敛的张力。千万千万不要出现类似主角定下目标，主角展望未来的句子。字数控制在5000字左右：

【章节大纲】
{outline_text}

【章节细纲】
{detail_text}

请开始写作正文：
"""  # ✍️ 可自定义 Prompt 模板

# 💡 Chapteroutlinedetailed outline(Chapter)
def match_chapter_files(outline_dir, detail_dir):
    outline_file = next((f for f in glob(os.path.join(outline_dir, '*.json')) if 'analysis' in f), None)
    detail_files = sorted(glob(os.path.join(detail_dir, '*.txt')))
    chapter_map = {}

    print("📄 找到细纲文件：")
    for f in detail_files:
        print("   -", os.path.basename(f))

    if outline_file:
        with open(outline_file, 'r', encoding='utf-8') as f:
            outline_data = json.load(f)
        for idx in CHAPTER_RANGE:
            chapter_id = f"{idx:03d}"
            detail_file = next((f for f in detail_files if f"_{idx}章" in os.path.basename(f) or f"第{idx}章" in os.path.basename(f) or f"{chapter_id}_" in os.path.basename(f)), None)
            if detail_file:
                chapter_map[str(idx)] = {
                    'outline': json.dumps(outline_data.get('characters', []), ensure_ascii=False, indent=2),
                    'detail': detail_file
                }
            else:
                print(f"⚠️ 第{idx}章未找到匹配的细纲文件，跳过。")
    return chapter_map

# ✨ OpenAI API
openai.api_key = API_KEY

def generate_chapter(chapter_num, outline_text, detail_text):
    prompt = PROMPT_TEMPLATE.format(
        chapter_num=chapter_num,
        outline_text=outline_text,
        detail_text=detail_text
    )
    client = openai.OpenAI(api_key=API_KEY)
    response = client.chat.completions.create(
        model="chatgpt-4o-latest",
  # model="gpt-4.1-2025-04-14",#
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content

# 🚀 ：、Chapter、save
def run_generation():
    print(f"📁 Outline 文件夹: {OUTLINE_DIR}")
    print(f"📁 Detail 文件夹: {DETAIL_DIR}")
    print(f"📕 输出书名: {BOOK_TITLE}")
    print(f"📚 生成章节范围: {CHAPTER_RANGE}\n")

    chapters = match_chapter_files(OUTLINE_DIR, DETAIL_DIR)
    print(f"🔍 匹配到章节数: {len(chapters)}")

    output_dir = f"/content/output/{BOOK_TITLE}"
    os.makedirs(output_dir, exist_ok=True)

    for idx in CHAPTER_RANGE:
        idx_str = str(idx)
        output_path = os.path.join(output_dir, f"第{idx}章.txt")

        if os.path.exists(output_path):
            print(f"✅ 第{idx}章已存在，跳过生成：{output_path}")
            continue

        if idx_str not in chapters:
            print(f"❌ 第{idx}章没有对应的大纲或细纲，跳过。")
            continue

        files = chapters[idx_str]
        print(f"➡️  第{idx}章 大纲预览: {files['outline'][:200]}...")
        print(f"➡️  第{idx}章 细纲路径: {files['detail']}")

        with open(files['detail'], 'r', encoding='utf-8') as f:
            detail_text = f.read()
        outline_text = files['outline']

        print(f"📖 正在生成第{idx}章...")
        try:
            content = generate_chapter(idx, outline_text, detail_text)
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(content)
            print(f"✅ 第{idx}章生成成功：{output_path}\n")
        except Exception as e:
            print(f"❌ 第{idx}章生成失败：{e}\n")
    print("生成完毕")
# ▶️
run_generation()

In [ ]:
API_KEY = "<YOUR_API_KEY>"  # 🔑 OpenAI API Key
OUTLINE_DIR = "/content/merged/《狩猎仙魔》作者_牧童听竹"  # 📁 outline( analysis.json )
DETAIL_DIR = "/content/json_results/《狩猎仙魔》作者_牧童听竹/001-005"    # 📁 detailed outline(detailed outline txt )

OUTLINE_DIR = "/content/merged/《大道纪》作者_裴屠狗"  # 📁 outline( analysis.json )
DETAIL_DIR = "/content/json_results/《大道纪》作者_裴屠狗/001-006"    # 📁 detailed outline(detailed outline txt )

In [ ]:
!rm -rf /content/output

In [ ]:
# ======================= ⚡️ChapteroutlineJSON - ⚡️ =======================

# Install dependencies
!pip install openai tqdm --quiet

import os
import json
import time
import openai
from tqdm import tqdm
from glob import glob

# ======================= =======================

API_KEY = "<YOUR_API_KEY>"          # 🔑 OpenAI Key
MODEL_NAME = "chatgpt-4o-latest"            # ✅ gpt-4o
TEMPERATURE = 0.2                # 🎯

OUTLINE_FOLDER = "/content/outlines"        # 📂 outlinetxt
OUTPUT_BASE_FOLDER = "/content/outputs"     # 📂 JSONoutput

# outlineChapter(，)
DEFAULT_TOTAL_CHAPTERS = 50

# ======================= OpenAI =======================

import openai

# client

def call_llm_generate_single_chapter_from_full_outline(full_outline_text, chapter_idx):
    PROMPT_TEMPLATE = """
    你是一位擅长网文结构化创作的作家，需要根据提供的【完整章节大纲】，只提取第{chapter_idx}章的内容，并独立生成标准JSON。所有的内容要尽可能丰富，紧凑，符合网络爽文的节奏。
    思考人物的所思，所感，所悟，所为，分析人物的关系，动力，目的，能力。

    这是基础中的基础，也是以人物为主要核心创作小说的一个入门技巧。

    然后是塑造人物，在塑造人物的过程中需要为人物分清主次，定位，意义。

    根据之前所分析出的结果，进行合理的整合，使人物圆融，饱满，灵动，立体。
    确保有趣、突然、合理、完整、通俗，又完全贴合给定的大纲。
    
     输出字段包括：
    - 章节定位导语
    - 情节摘要导语（不少于200字）
    - 情感与节奏导语
    - 关键场景分析
    - 人物角色变化
    - 情感张力变化
    - 节奏与结构观察
    - 出现人物（列表）
    - 出现道具（列表）
    - 出现场景（列表）
    - 伏笔_设下
    - 伏笔_回收
    
     要求：
    - 只处理第{chapter_idx}章！不要其他章节！
    - 严格输出【一个标准JSON对象】，不要数组，不要注释，不要其他文字！
    
    【完整章节大纲】
    {full_outline_text}
    
    请从完整大纲中提取【第{chapter_idx}章】，并开始输出标准JSON对象！
    """
    
    prompt = PROMPT_TEMPLATE.format(
        chapter_idx=chapter_idx,
        full_outline_text=full_outline_text,
    )


    # prompt = "？。"
    # response = client.chat.completions.create(
    # model=MODEL_NAME,
    # messages=[{"role": "user", "content": prompt}],
    # temperature=0.1,
    # max_tokens=50,
    # timeout=15, # ，
    # )
    # reply = response.choices[0].message.content.strip()
    # print(reply)
    # return reply
    client = openai.OpenAI(api_key=API_KEY)
    response = client.chat.completions.create(
        model="chatgpt-4o-latest",
# model="gpt-4.1-2025-04-14",#
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )
    
    return response.choices[0].message.content



# ======================= Chapter =======================

def generate_novel_by_looping_chapters(outline_file, novel_name, total_chapters):
    with open(outline_file, 'r', encoding='utf-8') as f:
        full_outline_text = f.read()

    print(f"📖 正在处理：{novel_name} ，共 {total_chapters} 章")

    novel_output_dir = os.path.join(OUTPUT_BASE_FOLDER, novel_name)
    os.makedirs(novel_output_dir, exist_ok=True)

    for chapter_idx in tqdm(range(1, total_chapters + 1), desc=f"保存 {novel_name}"):
        try:
            raw_text = call_llm_generate_single_chapter_from_full_outline(full_outline_text, chapter_idx)

            if not raw_text.strip():
                raise ValueError("⚠️ LLM返回空内容！")

        except Exception as e:
            print(f"❌ 第{chapter_idx}章出错：{e}")
            break
            continue

        save_path = os.path.join(novel_output_dir, f"chapter_{chapter_idx:03d}.txt")
        with open(save_path, 'w', encoding='utf-8') as f:
            f.write(raw_text)

    print(f"✅ 成功保存 {total_chapters} 章到 {novel_output_dir}\n")


# ======================= outline =======================

def batch_generate_all():
    outline_files = sorted(glob(os.path.join(OUTLINE_FOLDER, '*.txt')))

    if not outline_files:
        print("❗️ 没有找到任何大纲文件，请检查 OUTLINE_FOLDER 路径")
        return

    print(f"📚 发现 {len(outline_files)} 个大纲文件，开始批量处理...\n")

    for file_path in outline_files:
        novel_name = os.path.splitext(os.path.basename(file_path))[0]
        generate_novel_by_looping_chapters(file_path, novel_name, DEFAULT_TOTAL_CHAPTERS)

    print("\n🎯 全部小说处理完成！")

# ======================= =======================

batch_generate_all()

In [ ]:
# ============================================================
# 🚀 Usage example CELL
# ============================================================
import sys, types

if "merge_utils" not in sys.modules:
    merge_utils = types.ModuleType("merge_utils")
    for n, obj in globals().items():
        if n in ["run_pipeline", "is_chunk_dir", "auto_group", "call_deepseek"]:
            setattr(merge_utils, n, obj)
    sys.modules["merge_utils"] = merge_utils

# ， reload
merge_utils.run_pipeline(
    root_dir="/content/new_merged",
    merge_dir="/content/new_new_merged",
    api_key="<YOUR_API_KEY>"
)

import outline_batch as ob

ob.export_outlines(
    root_dir="/content/new_merged",          # JSON
    out_root="/content/outline_summaries",     # output
    fields=("情节摘要导语", "章节定位导语")      # detailed outline
)

In [67]:
# 📚 NovelChaptergenerator(outline+detailed outline，outputcontenttxt)
import os
import openai
import json
from glob import glob
from datetime import datetime

# ✅
API_KEY = "<YOUR_API_KEY>"
OUTLINE_DIR = "/content/new_new_merged/试一试的成果，我的直播间火爆阴阳大纲"  # 📁 outline( analysis.json)
DETAIL_DIR = "/content/new_merged/试一试的成果，我的直播间火爆阴阳大纲"    # 📁 detailed outline(txt)

BOOK_TITLE = "试一试的成果，我的直播间火爆阴阳大纲"
CHAPTER_RANGE = list(range(1, 20))  # Chapter

# 🧠 Prompt(content)
PROMPT_TEMPLATE = """
你是一位擅长网络小说创作的作家，请根据以下内容创作第{chapter_num}章的正文内容。

要求：
- 必须完全依据大纲和细纲内容，不要自行发挥。
- 严禁出现总结、展望未来、主角立志等情节。
- 风格要求：网文风格，语言自然有感染力，心理描写通过动作和细节体现，不要直接旁白。
- 避免“太小白”或“太装逼”的语言。
- 保持一定的内敛张力，节奏自然流畅。
- 字数建议5000字左右。

【章节大纲】
{outline_text}

【章节细纲】
{detail_text}

请根据以上内容，完整创作第{chapter_num}章正文：
"""

# ✨ Chapteroutlinedetailed outline
def match_chapter_files(outline_dir, detail_dir):
    outline_file = next((f for f in glob(os.path.join(outline_dir, '*.json')) if 'analysis' in f), None)
    detail_files = sorted(glob(os.path.join(detail_dir, '*.txt')))
    chapter_map = {}

    if outline_file:
        with open(outline_file, 'r', encoding='utf-8') as f:
            outline_data = json.load(f)

        outline_text = json.dumps(outline_data, ensure_ascii=False, indent=2)

        for idx in CHAPTER_RANGE:
            chapter_id = f"{idx:03d}"
            detail_file = next((f for f in detail_files if f"_{idx}章" in os.path.basename(f) or f"第{idx}章" in os.path.basename(f) or f"{chapter_id}_" in os.path.basename(f)), None)
            if detail_file:
                chapter_map[str(idx)] = {
                    'outline': outline_text,
                    'detail': detail_file
                }
            else:
                print(f"⚠️ 第{idx}章未找到细纲，跳过。")
    return chapter_map

# ✨ OpenAI APIcontent
openai.api_key = API_KEY

def generate_chapter_text(chapter_num, outline_text, detail_text):
    prompt = PROMPT_TEMPLATE.format(
        chapter_num=chapter_num,
        outline_text=outline_text,
        detail_text=detail_text
    )
    client = openai.OpenAI(api_key=API_KEY)
    response = client.chat.completions.create(
        model="chatgpt-4o-latest",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

# 🚀 ：++save
def run_generation():
    print(f"📁 Outline目录: {OUTLINE_DIR}")
    print(f"📁 Detail目录: {DETAIL_DIR}")
    print(f"📕 输出书名: {BOOK_TITLE}")
    print(f"📚 生成章节范围: {CHAPTER_RANGE}\n")

    chapters = match_chapter_files(OUTLINE_DIR, DETAIL_DIR)
    print(f"🔍 匹配到章节数: {len(chapters)}\n")

    output_dir = f"/content/output/{BOOK_TITLE}"
    os.makedirs(output_dir, exist_ok=True)

    for idx in CHAPTER_RANGE:
        idx_str = str(idx)
        output_path = os.path.join(output_dir, f"第{idx}章.txt")

        if os.path.exists(output_path):
            print(f"✅ 第{idx}章已存在，跳过：{output_path}")
            continue

        if idx_str not in chapters:
            print(f"❌ 第{idx}章无大纲或细纲，跳过。")
            continue

        files = chapters[idx_str]
        print(f"➡️ 正在生成第{idx}章...")

        with open(files['detail'], 'r', encoding='utf-8') as f:
            detail_text = f.read()
        outline_text = files['outline']

        try:
            content = generate_chapter_text(idx, outline_text, detail_text)
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(content)
            print(f"✅ 第{idx}章生成成功：{output_path}\n")
        except Exception as e:
            print(f"❌ 第{idx}章生成失败：{e}\n")

    print("🎯 全部章节生成完毕！")

# ▶️
run_generation()

📁 Outline目录: /content/new_new_merged/试一试的成果，我的直播间火爆阴阳大纲
📁 Detail目录: /content/new_merged/试一试的成果，我的直播间火爆阴阳大纲
📕 输出书名: 试一试的成果，我的直播间火爆阴阳大纲
📚 生成章节范围: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

⚠️ 第1章未找到细纲，跳过。
⚠️ 第2章未找到细纲，跳过。
⚠️ 第3章未找到细纲，跳过。
⚠️ 第4章未找到细纲，跳过。
⚠️ 第5章未找到细纲，跳过。
⚠️ 第6章未找到细纲，跳过。
⚠️ 第7章未找到细纲，跳过。
⚠️ 第8章未找到细纲，跳过。
⚠️ 第9章未找到细纲，跳过。
⚠️ 第10章未找到细纲，跳过。
⚠️ 第11章未找到细纲，跳过。
⚠️ 第12章未找到细纲，跳过。
⚠️ 第13章未找到细纲，跳过。
⚠️ 第14章未找到细纲，跳过。
⚠️ 第15章未找到细纲，跳过。
⚠️ 第16章未找到细纲，跳过。
⚠️ 第17章未找到细纲，跳过。
⚠️ 第18章未找到细纲，跳过。
⚠️ 第19章未找到细纲，跳过。
🔍 匹配到章节数: 0

❌ 第1章无大纲或细纲，跳过。
❌ 第2章无大纲或细纲，跳过。
❌ 第3章无大纲或细纲，跳过。
❌ 第4章无大纲或细纲，跳过。
❌ 第5章无大纲或细纲，跳过。
❌ 第6章无大纲或细纲，跳过。
❌ 第7章无大纲或细纲，跳过。
❌ 第8章无大纲或细纲，跳过。
❌ 第9章无大纲或细纲，跳过。
❌ 第10章无大纲或细纲，跳过。
❌ 第11章无大纲或细纲，跳过。
❌ 第12章无大纲或细纲，跳过。
❌ 第13章无大纲或细纲，跳过。
❌ 第14章无大纲或细纲，跳过。
❌ 第15章无大纲或细纲，跳过。
❌ 第16章无大纲或细纲，跳过。
❌ 第17章无大纲或细纲，跳过。
❌ 第18章无大纲或细纲，跳过。
❌ 第19章无大纲或细纲，跳过。
🎯 全部章节生成完毕！


In [ ]:
# 📚 Only passoutline+detailed outlineLLM，LLMcontent
import os
import openai
import json
from glob import glob
from datetime import datetime

# ✅
API_KEY = "<YOUR_API_KEY>"
OUTLINE_DIR = "/content/new_new_merged/试一试的成果，我的直播间火爆阴阳大纲"  # 📁 outline
DETAIL_DIR = "/content/new_merged/试一试的成果，我的直播间火爆阴阳大纲"    # 📁 detailed outline

BOOK_TITLE = "试一试的成果，我的直播间火爆阴阳大纲"
CHAPTER_RANGE = list(range(1, 20))  # Chapter

# 🧠 Prompt(outline+detailed outline)
PROMPT_TEMPLATE = """
你是一位擅长创作百万字长篇网络小说、兼具传统文学笔法的高级作家。

以下是整本小说的大纲和第{chapter_num}章节的细纲，请根据这些内容，完整创作第{chapter_num}章的正文。

【整本小说大纲】
{outline_text}

【第{chapter_num}章节细纲】
{detail_text}

⚠️ 重要创作要求：

- 本章节属于一部长篇小说中普通的一章，必须自然地嵌入整体叙事，不独立成篇。
- 禁止出现任何形式的总结、升华、展望未来。
- 禁止在结尾暗示后续剧情，如：
  - 不许使用“眸中燃起战意”“背后潜藏着黑影”“远处阴云涌动”等意象隐喻收尾。
  - 不许在最后一段描述环境变化（如风声、黑暗、呢喃声等）暗示接下来的危险或事件。
  - 不许通过角色内心的微妙变化（比如坚定、决心、战意）作为章节收尾。
- 正文必须自然以剧情进行至一个节点处停止，比如：
  - 完成当前事件的动作。
  - 完成一次对话。
  - 完成一个场景变化（但不强行升华）。
- 语言要求：
  - 叙事节奏符合网络小说爽感要求，高潮不断。
  - 描写细腻，动作心理自然流动，避免空洞直白的叙述。
  - 画面感强，但简洁有力，不堆砌辞藻。
- 人物要求：
  - 行动和情绪有合理动机，不为情节服务而做出牵强举动。
  - 角色行为和情绪反应必须自然真实，符合人性逻辑。
- 字数在5000字左右。

请严格遵守以上要求，直接撰写第{chapter_num}章的正文，不要添加任何额外说明。
"""



# ✨ outline
def load_outline(outline_dir):
    outline_file = next((f for f in glob(os.path.join(outline_dir, '*.json')) if 'analysis' in f), None)
    if outline_file:
        with open(outline_file, 'r', encoding='utf-8') as f:
            outline_data = json.load(f)
        outline_text = json.dumps(outline_data, ensure_ascii=False, indent=2)
        return outline_text
    else:
        raise FileNotFoundError("未找到 analysis.json 文件！")

# ✨ OpenAIcontent
openai.api_key = API_KEY

def generate_chapter_text(chapter_num, outline_text, detail_text):
    prompt = PROMPT_TEMPLATE.format(
        chapter_num=chapter_num,
        outline_text=outline_text,
        detail_text=detail_text
    )
    client = openai.OpenAI(api_key=API_KEY)
    response = client.chat.completions.create(
        model="chatgpt-4o-latest",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

# 🚀
def run_generation():
    outline_text = load_outline(OUTLINE_DIR)

    output_dir = f"/content/output/{BOOK_TITLE}"
    os.makedirs(output_dir, exist_ok=True)

    print(f"📚 开始生成小说《{BOOK_TITLE}》")

    for idx in CHAPTER_RANGE:
        output_path = os.path.join(output_dir, f"第{idx}章.txt")

        if os.path.exists(output_path):
            print(f"✅ 第{idx}章已存在，跳过：{output_path}")
            continue

        detail_file_candidates = glob(os.path.join(DETAIL_DIR, '**', f'{idx:03d}_*.txt'), recursive=True)

        if not detail_file_candidates:
            print(f"❌ 第{idx}章未找到细纲，跳过。")
            continue

        detail_path = detail_file_candidates[0]
        with open(detail_path, 'r', encoding='utf-8') as f:
            detail_text = f.read()

        print(f"📖 正在生成第{idx}章...")

        try:
            content = generate_chapter_text(idx, outline_text, detail_text)
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(content)
            print(f"✅ 第{idx}章保存成功：{output_path}\n")
        except Exception as e:
            print(f"❌ 第{idx}章生成失败：{e}\n")

    print("🎯 全部章节生成完毕！")

# ▶️
run_generation()

In [89]:
# 📚 NovelChapterrefinement system - outlineTXT + content + + content

import os
import openai
from glob import glob
from tqdm import tqdm

# ✅ ()
API_KEY = "<YOUR_API_KEY>"
OUTLINE_DIR = "/content/outlines/试一试的成果，我的直播间火爆阴阳大纲.txt"      # analysis.json
ROUGH_DIR = "/content/output/试一试的成果，我的直播间火爆阴阳大纲"         # Chaptertxt(X)
FINAL_DIR = "/content/最终精修保存路径"       # saveChapter
BOOK_TITLE = "试一试的成果，我的直播间火爆阴阳大纲"
CHAPTER_RANGE = list(range(1, 8))           # Chapter，1-19

openai.api_key = API_KEY

# ✨ outline(TXT)
def load_outline(outline_path):
    if not os.path.exists(outline_path):
        raise FileNotFoundError(f"未找到大纲文件: {outline_path}")
    
    with open(outline_path, 'r', encoding='utf-8') as f:
        outline_text = f.read().strip()
    
    return outline_text

# ✨ Prompt

def build_rewrite_prompt(outline_text, chapter_num, chapter_text, prev_chapter_text=None, next_chapter_text=None):
    prompt = f"""
你是一位擅长创作百万字长篇网络小说、同时兼具传统文学叙事功底的高级作家。

以下提供了：

【整本小说大纲】
{outline_text}
"""

    if prev_chapter_text:
        prompt += f"""

【第{chapter_num-1}章正文（上一章）】
{prev_chapter_text}
"""

    prompt += f"""

【第{chapter_num}章初版正文（需要修订）】
{chapter_text}
"""

    if next_chapter_text:
        prompt += f"""

【第{chapter_num+1}章正文（下一章）】
{next_chapter_text}
"""

    prompt += f"""

⚠️ 请根据以上内容，对【第{chapter_num}章初版正文】进行**深入重新写作**，严格遵循以下要求：

【连贯性】
- 本章必须自然衔接前一章情节，并顺畅过渡至下一章设定。
- 保持整体剧情流畅，确保上一章与下一章之间没有冲突或情绪断层。
- 连贯性优先于其他因素，是最高要求。

【大纲一致性】
- 本章的剧情应严格遵循整本小说大纲设定。
- 允许根据连贯性优化具体细节，但不得偏离大纲主干走向。

【写作规范】
- 本章节是长篇小说中自然的一部分，不应独立成篇。
- 禁止在章节结尾进行总结、升华或展望未来。
- 禁止使用任何暗示式环境隐喻，如：
  - “眸中燃起战意”
  - “黑影蠢蠢欲动”
  - “远处传来呢喃低语”
- 禁止通过角色心理变化（如坚定、战意觉醒）来强行收尾。
- 章节结尾应自然停留在当前冲突、对话或场景动作节点上。

【篇幅与风格】
- 字数控制在约5000字左右。
- 允许对原章节进行较大幅度的调整或重写，只要整体符合大纲与上下文。
- 叙事风格保持流畅自然，兼具网络小说的节奏感与传统文学的细腻感。

请根据以上要求，**重新撰写并优化第{chapter_num}章正文**，不添加任何解释或总结。
""".replace("{chapter_num}", str(chapter_num))


    return prompt


# ✨ OpenAI
def call_openai(prompt):
    client = openai.OpenAI(api_key=API_KEY)
    response = client.chat.completions.create(
        model="chatgpt-4o-latest",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5
    )
    return response.choices[0].message.content.strip()

# 🚀
def run_rewrite_all():
    outline_text = load_outline(OUTLINE_PATH)
    os.makedirs(FINAL_DIR, exist_ok=True)

    print(f"📚 正在精修小说《{BOOK_TITLE}》...")
    print(f"🔎 章节范围: {CHAPTER_RANGE}\n")

    for idx in tqdm(CHAPTER_RANGE):
        chapter_num = idx
        rough_path = os.path.join(ROUGH_DIR, f"第{chapter_num}章.txt")
        final_path = os.path.join(FINAL_DIR, f"第{chapter_num}章_精修版.txt")

        if os.path.exists(final_path):
            print(f"✅ 第{chapter_num}章已精修，跳过")
            continue

        # Chapter
        if not os.path.exists(rough_path):
            print(f"❌ 第{chapter_num}章初版不存在，跳过")
            continue
        with open(rough_path, 'r', encoding='utf-8') as f:
            chapter_text = f.read()

        # Chapter()
        prev_chapter_text = None
        if chapter_num > 1:
            prev_path = os.path.join(FINAL_DIR, f"第{chapter_num-1}章_精修版.txt")
            if os.path.exists(prev_path):
                with open(prev_path, 'r', encoding='utf-8') as f:
                    prev_chapter_text = f.read()

        # Chapter()
        next_chapter_text = None
        next_path = os.path.join(ROUGH_DIR, f"第{chapter_num+1}章.txt")
        if os.path.exists(next_path):
            with open(next_path, 'r', encoding='utf-8') as f:
                next_chapter_text = f.read()

        # Prompt
        prompt = build_rewrite_prompt(
            outline_text=outline_text,
            chapter_num=chapter_num,
            chapter_text=chapter_text,
            prev_chapter_text=prev_chapter_text,
            next_chapter_text=next_chapter_text
        )

        # LLM
        print(f"🛠️ 正在精修第{chapter_num}章...")
        try:
            final_content = call_openai(prompt)

            # save
            with open(final_path, 'w', encoding='utf-8') as f:
                f.write(final_content)
            print(f"✅ 第{chapter_num}章精修完成并保存！\n")
        except Exception as e:
            print(f"❌ 第{chapter_num}章精修失败：{e}\n")

    print("\n🎯 全部章节精修完毕！")

# ▶️
run_rewrite_all()

📚 正在精修小说《试一试的成果，我的直播间火爆阴阳大纲》...
🔎 章节范围: [1, 2, 3, 4, 5, 6, 7]



  0%|          | 0/7 [00:00<?, ?it/s]

🛠️ 正在精修第1章...


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 14%|█▍        | 1/7 [00:22<02:13, 22.27s/it]

✅ 第1章精修完成并保存！

🛠️ 正在精修第2章...


...

✅ 第5章精修完成并保存！

🛠️ 正在精修第6章...


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO: Retrying request to /chat/completions in 4.976000 seconds
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 86%|████████▌ | 6/7 [02:26<00:26, 26.89s/it]

✅ 第6章精修完成并保存！

🛠️ 正在精修第7章...


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO: Retrying request to /chat/completions in 12.546000 seconds
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
100%|██████████| 7/7 [03:05<00:00, 26.47s/it]

✅ 第7章精修完成并保存！


🎯 全部章节精修完毕！


In [ ]:
# 📚 NovelChapterrefinement system - outlineTXT + content + + content

import os
import openai
from glob import glob
from tqdm import tqdm

# ✅ ()
API_KEY = "<YOUR_API_KEY>"
OUTLINE_DIR = "/content/outlines/试一试的成果，我的直播间火爆阴阳大纲.txt"      # analysis.json
ROUGH_DIR = "/content/output/试一试的成果，我的直播间火爆阴阳大纲"         # Chaptertxt(X)
FINAL_DIR = "/content/最终精修保存路径"       # saveChapter
BOOK_TITLE = "试一试的成果，我的直播间火爆阴阳大纲"
CHAPTER_RANGE = list(range(1, 8))           # Chapter，1-19

openai.api_key = API_KEY

# ✨ outline(TXT)
def load_outline(outline_path):
    if not os.path.exists(outline_path):
        raise FileNotFoundError(f"未找到大纲文件: {outline_path}")
    
    with open(outline_path, 'r', encoding='utf-8') as f:
        outline_text = f.read().strip()
    
    return outline_text

# ✨ Prompt

def build_rewrite_prompt(outline_text, chapter_num, chapter_text, prev_chapter_text=None, next_chapter_text=None):
    prompt = f"""
你是一位擅长创作百万字长篇网络小说、同时兼具传统文学叙事功底的高级作家。

以下提供了：

【整本小说大纲】
{outline_text}
"""

    if prev_chapter_text:
        prompt += f"""

【第{chapter_num-1}章正文（上一章）】
{prev_chapter_text}
"""

    prompt += f"""

【第{chapter_num}章初版正文（需要修订）】
{chapter_text}
"""

    if next_chapter_text:
        prompt += f"""

【第{chapter_num+1}章正文（下一章）】
{next_chapter_text}
"""

    prompt += f"""

⚠️ 请根据以上内容，对【第{chapter_num}章初版正文】进行**深入重新写作**，严格遵循以下要求：

【连贯性】
- 本章必须自然衔接前一章情节，并顺畅过渡至下一章设定。
- 保持整体剧情流畅，确保上一章与下一章之间没有冲突或情绪断层。
- 连贯性优先于其他因素，是最高要求。

【大纲一致性】
- 本章的剧情应严格遵循整本小说大纲设定。
- 允许根据连贯性优化具体细节，但不得偏离大纲主干走向。

【写作规范】
- 本章节是长篇小说中自然的一部分，不应独立成篇。
- 禁止在章节结尾进行总结、升华或展望未来。
- 禁止使用任何暗示式环境隐喻，如：
  - “眸中燃起战意”
  - “黑影蠢蠢欲动”
  - “远处传来呢喃低语”
- 禁止通过角色心理变化（如坚定、战意觉醒）来强行收尾。
- 章节结尾应自然停留在当前冲突、对话或场景动作节点上。

【篇幅与风格】
- 字数控制在约5000字左右。
- 允许对原章节进行较大幅度的调整或重写，只要整体符合大纲与上下文。
- 叙事风格保持流畅自然，兼具网络小说的节奏感与传统文学的细腻感。

请根据以上要求，**重新撰写并优化第{chapter_num}章正文**，不添加任何解释或总结。
""".replace("{chapter_num}", str(chapter_num))


    return prompt


# ✨ OpenAI
def call_openai(prompt):
    client = openai.OpenAI(api_key=API_KEY)
    response = client.chat.completions.create(
        model="chatgpt-4o-latest",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5
    )
    return response.choices[0].message.content.strip()

# 🚀
def run_rewrite_all():
    outline_text = load_outline(OUTLINE_PATH)
    os.makedirs(FINAL_DIR, exist_ok=True)

    print(f"📚 正在精修小说《{BOOK_TITLE}》...")
    print(f"🔎 章节范围: {CHAPTER_RANGE}\n")

    for idx in tqdm(CHAPTER_RANGE):
        chapter_num = idx
        rough_path = os.path.join(ROUGH_DIR, f"第{chapter_num}章.txt")
        final_path = os.path.join(FINAL_DIR, f"第{chapter_num}章_精修版.txt")

        if os.path.exists(final_path):
            print(f"✅ 第{chapter_num}章已精修，跳过")
            continue

        # Chapter
        if not os.path.exists(rough_path):
            print(f"❌ 第{chapter_num}章初版不存在，跳过")
            continue
        with open(rough_path, 'r', encoding='utf-8') as f:
            chapter_text = f.read()

        # Chapter()
        prev_chapter_text = None
        if chapter_num > 1:
            prev_path = os.path.join(FINAL_DIR, f"第{chapter_num-1}章_精修版.txt")
            if os.path.exists(prev_path):
                with open(prev_path, 'r', encoding='utf-8') as f:
                    prev_chapter_text = f.read()

        # Chapter()
        next_chapter_text = None
        next_path = os.path.join(ROUGH_DIR, f"第{chapter_num+1}章.txt")
        if os.path.exists(next_path):
            with open(next_path, 'r', encoding='utf-8') as f:
                next_chapter_text = f.read()

        # Prompt
        prompt = build_rewrite_prompt(
            outline_text=outline_text,
            chapter_num=chapter_num,
            chapter_text=chapter_text,
            prev_chapter_text=prev_chapter_text,
            next_chapter_text=next_chapter_text
        )

        # LLM
        print(f"🛠️ 正在精修第{chapter_num}章...")
        try:
            final_content = call_openai(prompt)

            # save
            with open(final_path, 'w', encoding='utf-8') as f:
                f.write(final_content)
            print(f"✅ 第{chapter_num}章精修完成并保存！\n")
        except Exception as e:
            print(f"❌ 第{chapter_num}章精修失败：{e}\n")

    print("\n🎯 全部章节精修完毕！")

# ▶️
run_rewrite_all()

In [ ]:
!rm -rf /content/new_merged

In [ ]:
API_KEY = "<YOUR_API_KEY>"
OUTLINE_DIR = "/content/outlines/试一试的成果，我的直播间火爆阴阳大纲.txt"      # analysis.json
ROUGH_DIR = "/content/output/试一试的成果，我的直播间火爆阴阳大纲"         # Chaptertxt(X)
FINAL_DIR = "/content/最终精修保存路径"       # saveChapter
BOOK_TITLE = "试一试的成果，我的直播间火爆阴阳大纲"
CHAPTER_RANGE = list(range(1, 8))           # Chapter，1-19

In [91]:
# 📚 Noveloutlinerewrite system - outlinericher、plot consistency、more rounded characters
import os
import openai

# ✅
API_KEY = "<YOUR_API_KEY>"

OUTLINE_PATH = "/content/outlines/试一试的成果，我的直播间火爆阴阳大纲.txt"   # outlinetxtpath
OUTPUT_PATH = "/content/outputs/大纲重写版.txt"                                           # outputoutlinepath

openai.api_key = API_KEY

# ✨ outline
def load_outline(outline_path):
    if not os.path.exists(outline_path):
        raise FileNotFoundError(f"未找到大纲文件: {outline_path}")
    with open(outline_path, 'r', encoding='utf-8') as f:
        outline_text = f.read().strip()
    return outline_text

# ✨ outlinePrompt
def build_outline_rewrite_prompt(outline_text):
    prompt = f"""
你是一位擅长创作百万字长篇网络小说的高级作家，兼具传统文学叙事功底与网文爽点技巧。

以下是一本小说的原始大纲，请根据要求进行全面重写优化：

【原始大纲】
{outline_text}

⚡ 重写要求：

- 保持原大纲的世界观、核心设定和基本方向不变。
- 使故事主线更加清晰明确，有明确的阶段性目标和剧情发展路线。
- 增加更多具有特色的人物（主角群、配角群、反派阵营），并简单标注主要人物性格与作用。
- 丰富情节细节，提升冲突张力、爽点爆发节奏，同时兼顾逻辑合理性。
- 加强人物成长线，让主角与主要配角在剧情中有明显性格变化与成长轨迹。
- 保持叙事节奏自然连贯，避免跳跃，确保因果链完整。
- 每一章节剧情之间要有合理过渡，确保故事从开端到高潮到后期稳定推进。
- 语言流畅自然，逻辑严谨，不需要任何解释或注释。

请根据以上要求，全面重写优化这个大纲，直接输出重写后的新版大纲正文，要求有每一章节的大纲,爽点，悬念，节奏。
"""
    return prompt

# ✨ OpenAIoutline
def call_openai(prompt):
    client = openai.OpenAI(api_key=API_KEY)
    response = client.chat.completions.create(
        model="chatgpt-4o-latest",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.6  # ，
    )
    return response.choices[0].message.content.strip()

# 🚀
def run_outline_rewrite():
    outline_text = load_outline(OUTLINE_PATH)

    print("📚 开始重写大纲...")

    prompt = build_outline_rewrite_prompt(outline_text)

    try:
        rewritten_outline = call_openai(prompt)

        # saveoutline
        os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
        with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
            f.write(rewritten_outline)

        print(f"✅ 大纲重写完成，已保存到：{OUTPUT_PATH}")

    except Exception as e:
        print(f"❌ 大纲重写失败：{e}")

# ▶️
run_outline_rewrite()

📚 开始重写大纲...


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ 大纲重写完成，已保存到：/content/outputs/大纲重写版.txt
